# 생성형 AI 기본 API 호출

## 1. 미리 준비

### 1.1 프로그램 설치

In [2]:
# 필요한 패키지 설치
! pip install --upgrade boto3  > /dev/null 2>&1
! pip list | egrep 'boto3' 

boto3                                    1.40.40


### 1.2 인증 선언

In [3]:
import boto3
import time
import json

# 프로그램의 시간측정을 위한 Class 선언 
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        print("\n\n-----------")
        print(f"모델 실행 시간: {time.time() - self.start:.2f}초 \n")
        
# AWS 자격 증명 확인
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    print("AWS 자격 증명 확인 완료")
    print(f"리전: {session.region_name}")
except Exception as e:
    print(f"AWS 자격 증명 오류: {e}")
    print("AWS CLI를 사용하여 자격 증명을 설정해주세요: aws configure")


AWS 자격 증명 확인 완료
리전: us-east-1


### 1.3 Bedrock 클라이언트 생성 

In [4]:
bedrock = session.client('bedrock-runtime')
bedrock2 = session.client('bedrock')

### 1.4 Bedrock이 제공하는 모델 확인 하기


In [5]:

response = bedrock2.list_foundation_models()

print("=== 모델 상세 정보 ===")
for model in response['modelSummaries']:
    print(f"모델 ID: {model['modelId']}")
    print(f"모델 ARN: {model['modelArn']}")
    print(f"제공업체: {model['providerName']}")
    print("=" * 60)


=== 모델 상세 정보 ===
모델 ID: stability.stable-image-remove-background-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/stability.stable-image-remove-background-v1:0
제공업체: Stability AI
모델 ID: stability.stable-image-style-guide-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/stability.stable-image-style-guide-v1:0
제공업체: Stability AI
모델 ID: stability.stable-image-control-sketch-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/stability.stable-image-control-sketch-v1:0
제공업체: Stability AI
모델 ID: anthropic.claude-sonnet-4-20250514-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-sonnet-4-20250514-v1:0
제공업체: Anthropic
모델 ID: stability.stable-image-erase-object-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/stability.stable-image-erase-object-v1:0
제공업체: Stability AI
모델 ID: stability.stable-image-control-structure-v1:0
모델 ARN: arn:aws:bedrock:us-east-1::foundation-model/stability.stable-image-control-structure-v1:0
제공업체: Stability AI
모델 ID:

### 1.5 이 프로그램에서 사용할 기본 모델 선언 

In [6]:
# 현재 리전 확인
current_region = boto3.Session().region_name

# 현재 리전에 맞는 Model ID 자동 검색
models = bedrock2.list_foundation_models()['modelSummaries']

base_model=""
# 버지니아 리전인 경우 Cross-region inference 사용
if current_region == 'us-east-1':
    base_model = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
else:
    find_models = next((m for m in models if 'claude-3-5-haiku' in m['modelId'].lower()), None)
    base_model = find_models['modelId']

print("[Model ID] claude-3-5-haiku :", base_model)
print("[Current Region]:", current_region)


[Model ID] claude-3-5-haiku : us.anthropic.claude-3-5-haiku-20241022-v1:0
[Current Region]: us-east-1


---
## 2. Bedrock Invoke API 호출 
- https://docs.aws.amazon.com/bedrock/latest/userguide/inference-invoke.html 
  

### 2.1 Claude model 용 Helper함수 선언후 모델 호출 

In [53]:
def test_invokeapi_claude(message, system="", history=None ,display=True):
    if history:
        history.append({"role": "user", "content": [{"type": "text", "text": message}]})
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1500,
            "temperature": 0.5,
            "system": system,
            "messages": history
            })
    else:
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1500,
            "temperature": 0.5,
            "system": system,
            "messages": [{"role": "user", "content": message}]
            })   

    with Timer():
        response = bedrock.invoke_model(
            modelId=base_model,
            # modelId="us.anthropic.claude-3-haiku-20240307-v1:0",
            body=body
        )

    response_body = json.loads(response['body'].read())
    output_text = response_body['content'][0]['text']
    
    if display:
        print(output_text)
        print("\n[모델 실행 정보] ------------------------------------------")
        for key in ['input_tokens', 'output_tokens']:
            print(f"{key} : {response_body.get('usage', {}).get(key, 0)}")

    return output_text

Ret=test_invokeapi_claude(message="김치 볶음밥은 어떻게 만들어야 하나요?", 
            system="당신은 한식 요리 전문가 입니다. 전문가로써의 정중한 답변을 해주세요.")



-----------
모델 실행 시간: 5.88초 

김치 볶음밥을 맛있게 만드는 방법을 알려드리겠습니다.

재료:
- 밥
- 김치
- 돼지고기 (선택사항)
- 계란
- 식용유
- 소금, 후추

조리 순서:
1. 김치를 잘게 썰어주세요
2. 팬에 식용유를 두르고 중불로 달군 후
3. 김치와 돼지고기를 볶아주세요
4. 밥을 넣고 함께 볶으면서 고루 섞어주세요
5. 마지막에 계란을 살짝 부어 볶아주세요
6. 소금, 후추로 간을 맞춰주세요

팁:
- 김치는 숙성된 김치가 더 맛있어요
- 밥은 차갑고 단단한 밥이 볶음밥에 좋습니다
- 참기름을 살짝 뿌리면 풍미가 더해집니다

[모델 실행 정보] ------------------------------------------
input_tokens : 71
output_tokens : 322


### 2.3 Claude 모델의 Stream 방식 모델 호출

In [46]:
def test_invokeapi_claude_stream(message, system=""):
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1500,
        "temperature": 0.5,
        "system": system,
        "messages": [{"role": "user", "content": message}]
    })

    with Timer():
        response = bedrock.invoke_model_with_response_stream(
            modelId=base_model,
            body=body
        )

    # 스트림 응답 처리
    stream = response.get("body")
    full_text = ""

    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                chunk_json = json.loads(chunk.get("bytes").decode())

                # 메시지 시작 처리
                if chunk_json.get("type") == "message_start":
                    continue

                # 콘텐츠 블록 시작 처리
                elif chunk_json.get("type") == "content_block_start":
                    continue

                # 콘텐츠 블록 델타 처리 (실제 텍스트)
                elif chunk_json.get("type") == "content_block_delta":
                    delta = chunk_json.get("delta", {})
                    if delta.get("type") == "text_delta":
                        delta_text = delta.get("text", "")
                        print(delta_text, end="", flush=True)
                        full_text += delta_text

                # 메시지 델타 처리 (사용량 정보)
                elif chunk_json.get("type") == "message_delta":
                    usage = chunk_json.get("usage")
                    if usage:
                        print("\n\n--모델 실행 정보 --------------------------------------------------------------")
                        print(f"inputTokens : {usage.get('input_tokens', 0)}")
                        print(f"outputTokens : {usage.get('output_tokens', 0)}")

    return full_text

# 사용 예시
test_invokeapi_claude_stream(
    message="김치 볶음밥은 어떻게 만들어야 하나요?",
    system="당신은 한식 요리 전문가 입니다. 전문가로써의 정중한 답변을 해주세요."
)




-----------
모델 실행 시간: 1.35초 

김치 볶음밥을 맛있게 만드는 방법을 알려드리겠습니다.

재료:
- 밥
- 김치
- 돼지고기 또는 햄
- 계란
- 파
- 참기름
- 간장
- 소금, 후추

조리 순서:
1. 김치를 잘게 다져주세요
2. 팬에 기름을 두르고 김치와 고기를 볶아주세요
3. 밥을 넣고 함께 볶으면서 간장으로 간을 맞춰주세요
4. 마지막에 계란을 살짝 프라이해서 올려주세요
5. 파를 다져 위에 뿌리고 참기름을 살짝 둘러주세요

팁: 밥은 차가운 밥이 더 맛있으며, 팬을 중불로 해주세요.

--모델 실행 정보 --------------------------------------------------------------
inputTokens : 0
outputTokens : 285


'김치 볶음밥을 맛있게 만드는 방법을 알려드리겠습니다.\n\n재료:\n- 밥\n- 김치\n- 돼지고기 또는 햄\n- 계란\n- 파\n- 참기름\n- 간장\n- 소금, 후추\n\n조리 순서:\n1. 김치를 잘게 다져주세요\n2. 팬에 기름을 두르고 김치와 고기를 볶아주세요\n3. 밥을 넣고 함께 볶으면서 간장으로 간을 맞춰주세요\n4. 마지막에 계란을 살짝 프라이해서 올려주세요\n5. 파를 다져 위에 뿌리고 참기름을 살짝 둘러주세요\n\n팁: 밥은 차가운 밥이 더 맛있으며, 팬을 중불로 해주세요.'

### 2.4 모델이 대화를 기억 하는지 테스트 
- 기본적으로 모델은 기억 기능이 없다!

#### 1) 모델에게 내가 누구 인지 정보를 준다

In [47]:
ret=test_invokeapi_claude(message="나는 조재구라고해 앞으로 나를 재구님 이라고 해줘!",display=False) 
print(ret)



-----------
모델 실행 시간: 1.41초 

네, 알겠습니다. 앞으로 재구님이라고 부르겠습니다. 무엇을 도와드릴까요?


#### 2) 내가 누구 인지 물어본다
- 모른다.

In [48]:
ret=test_invokeapi_claude(message="나에 이름이 무엇이라고 했지?",display=False) 
print(ret)



-----------
모델 실행 시간: 1.80초 

제가 당신의 이름을 들은 적이 없습니다. 제 이름은 Claude입니다.


### 2.5 모델이 대화의 내용을 기억하게 한다.

#### 1) 대화를 기억할 저장소를 만든다 
- 이 저장소에 모델에게 입력한 내용과 출력한 내용을 모두 기록하도록 한다.
- List 구조 방식으로 기록 한다.

In [64]:
# 기억 장소를 하나 만든다.
history = []

#### 2) 첫 번째 질문과 답변을 얻는다.

In [ ]:
# 첫번째 질문과 응답 
MessageNumber1="나에 이름은  조재구 라고 한다. 앞으로 나를 재구님 이라고 불러줘!"
print("\n[질문1]",MessageNumber1)
assistant_messages=test_invokeapi_claude(message=MessageNumber1,history=history,display=False) 
print("[응답 1번째:] -----------------------")
print(assistant_messages)


[질문1] 나에 이름은  조재구 라고 한다. 앞으로 나를 재구님 이라고 불러줘!


-----------
모델 실행 시간: 1.28초 


[응답 1번째:] -----------------------
알겠습니다, 재구님. 앞으로 재구님이라고 부르겠습니다.


#### 3) 첫 번째 대화 내용을 기억장소에 기록한다.
- 모델에게 요청한 정보와 응답한 메세지를 기록 한다.

In [ ]:
# 응답을 기억 장소에 저장한다. 
history.append({"role": "assistant", "content": [{"type": "text", "text": assistant_messages}]})


#### 4) 두번째 질문을 한다. 이때 기억한 대화 정보도 함께 요청한다.
- 아래 요청할때 history=history 라는 변수 선언이 보입니다. 위에 Helper 함수에 histoy 값을 이용해서 Body를 만들도록 하고 있습니다.
- 이번에 응답에서 "재구님" 이라고 정확하게 기억하는 것을 알수 있음.

In [ ]:
# 두번째 질문을 진행한다.
MessageNumber2="나에 이름이 무엇이라고 했지?"
print("\n[질문2]",MessageNumber2)
print("[응답 2번째:] -----------------------")
assistant_messages=test_invokeapi_claude(message=MessageNumber1,history=history,display=False) 
print(assistant_messages)


[질문2] 나에 이름이 무엇이라고 했지?

[응답 2번째:] -----------------------


-----------
모델 실행 시간: 1.14초 

알겠습니다, 재구님. 앞으로 재구님이라고 부르겠습니다.


#### 5) 두번째 대화 내용도 기억장소에 기록한다.

In [68]:
# 질문과 응답을 기억 장소에 추가 저장한다. 
history.append({"role": "user", "content": [{"type": "text", "text": MessageNumber2}]})
history.append({"role": "assistant", "content": [{"type": "text", "text": assistant_messages}]})

#### 6) 기억 장소에 기록된 내용을 확인한다.

In [69]:
# 현재 기억장소의 내용을 표시한다.
for item in history:
    print(item)
print("\n")

{'role': 'user', 'content': [{'type': 'text', 'text': '나에 이름은  조재구 라고 한다. 앞으로 나를 재구님 이라고 불러줘!'}]}
{'role': 'assistant', 'content': [{'type': 'text', 'text': '알겠습니다, 재구님. 앞으로 재구님이라고 부르겠습니다.'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': '나에 이름은  조재구 라고 한다. 앞으로 나를 재구님 이라고 불러줘!'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': '나에 이름이 무엇이라고 했지?'}]}
{'role': 'assistant', 'content': [{'type': 'text', 'text': '알겠습니다, 재구님. 앞으로 재구님이라고 부르겠습니다.'}]}




## 3. Converse API 
- 대화를 기능을 편리하게 제공되는 API (Chatbot선언)
- 모델마다 다르게 Invoke API를 사용해야 하는 것과 다르게 통일된 API를 사용함 
- https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html

### 3.1 Parameter 선언

In [70]:
# System prompt 선언
system_prompts = [{"text": "당신은 한식 요리 전문가 입니다. 전문가로써의 정중한 답변을 해주세요."}]

# 기억 기능을 위한 연속적인 API 호출과 출력의 메세지들을 위한 
messages = []

# 기본 Inference parameters 선언 , 모델마다 값이 다를수 있음. 
inference_config = {"temperature": 0.5}

# 추가적인 Inference parameters, 모델마다 값이 다를수 있음. 
additional_model_fields = {"top_p": 0.9}


### 3.2 Converse API 로 모델 호출 - 첫번째 질의

In [71]:
# 첫번째 질문을 
messages.append( 
    {
        "role": "user",
        "content": [{"text": "김치 찌개를 끓이고 싶어요! 레시피를 알려주세요"}]
    }
)

response = bedrock.converse(
        modelId=base_model,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

output_message = response['output']['message']

# 출력된 데이터를 첫번째 질문의 메세지 영역에 추가 한다.
messages.append(output_message)
print(output_message['content'][0]['text'])

print("------------------ MetaData ----------------------------------")
Usage=response.get("usage")
for key, value in Usage.items():
    print(f"{key}: {value}")

안녕하세요! 김치 찌개는 한국인들이 가장 사랑하는 대표적인 요리 중 하나입니다. 정통 김치 찌개 레시피를 알려드리겠습니다.

재료:
- 김치 1컵 (잘 익은 김치)
- 돼지고기 200g (삼겹살 또는 목살)
- 두부 1모
- 대파 1대
- 고춧가루, 국간장, 다진 마늘

조리 순서:
1. 김치를 적당한 크기로 잘라주세요.
2. 팬에 돼지고기를 살짝 볶아주세요.
3. 김치를 넣고 함께 볶아주세요.
4. 물을 붓고 끓이면서 간을 맞춰주세요.
5. 두부와 대파를 넣고 마무리합니다.

팁: 김치는 잘 익은 것을 사용하면 더욱 맛있습니다.

맛있게 드세요!
------------------ MetaData ----------------------------------
inputTokens: 76
outputTokens: 335
totalTokens: 411
cacheReadInputTokens: 0
cacheWriteInputTokens: 0


### 3.3 Converse API 로 모델 호출 - 두번째 질의
- 주의 : 모델 실행후 Input 토큰수를 확인해야함,
- 첫번째 모델에게 입력/출력 값을 두번째 모델의 입력으로 전달함.

```mermaid
flowchart TD
    Input[첫 번째 질문] --> Model1[모델 호출]
    Model1 --> Output1[첫 번째 모델 출력]
    
    Input --> Combine[첫번째 질문 + 답 + 두번째 질문 입력 결합 ]
    Output1 --> Combine
    
    Combine --> Model2[두 번째 모델]
    Model2 --> FinalOutput[최종 출력]
    
    style Input fill:#42a5f5,color:#fff
    style Model1 fill:#ab47bc,color:#fff
    style Output1 fill:#ffa726,color:#000
    style Combine fill:#ffee58,color:#000
    style Model2 fill:#ab47bc,color:#fff
    style FinalOutput fill:#66bb6a,color:#fff
```



In [72]:
# 앞선 메세지에 2번째 질문을 추가한다. 
messages.append(  
   {
        "role": "user",
        "content": [{"text": "이 레시피에 들이가는 영양성분을 알려주세요"}]
    }
)

response = bedrock.converse(
        modelId=base_model,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

output_message = response['output']['message']

# 출력된 데이터를 메세지 영역에 추가 한다.
messages.append(output_message)
print(output_message['content'][0]['text'])

print("------------------ MetaData ----------------------------------")
Usage=response.get("usage")
for key, value in Usage.items():
    print(f"{key}: {value}")

김치 찌개의 주요 영양성분을 상세히 알려드리겠습니다.

🥬 영양성분 (1인분 기준):
- 칼로리: 약 250-300kcal
- 단백질: 15-20g
- 지방: 15-18g
- 탄수화물: 10-15g

🌟 주요 영양소:
1. 단백질 
- 돼지고기: 근육 발달, 면역력 강화
- 두부: 식물성 단백질 공급

2. 비타민
- 비타민 A, C: 면역력 증진
- 비타민 B군: 대사 활성화

3. 무기질
- 칼슘: 뼈 건강
- 철분: 혈액 순환
- 칼륨: 혈압 조절

4. 김치의 특별한 영양
- 프로바이오틱스: 장 건강 개선
- 항산화 성분: 노화 방지

건강하고 맛있는 한 끼 식사가 되시길 바랍니다!
------------------ MetaData ----------------------------------
inputTokens: 438
outputTokens: 339
totalTokens: 777
cacheReadInputTokens: 0
cacheWriteInputTokens: 0


### 3.4 대화 기록 확인 

In [73]:
for message in messages:
            print(f"Role: {message['role']}")
            print("-----------------")
            for content in message['content']:
                print(f"Text: {content['text']}")
                print("-" * 40)
            print()


Role: user
-----------------
Text: 김치 찌개를 끓이고 싶어요! 레시피를 알려주세요
----------------------------------------

Role: assistant
-----------------
Text: 안녕하세요! 김치 찌개는 한국인들이 가장 사랑하는 대표적인 요리 중 하나입니다. 정통 김치 찌개 레시피를 알려드리겠습니다.

재료:
- 김치 1컵 (잘 익은 김치)
- 돼지고기 200g (삼겹살 또는 목살)
- 두부 1모
- 대파 1대
- 고춧가루, 국간장, 다진 마늘

조리 순서:
1. 김치를 적당한 크기로 잘라주세요.
2. 팬에 돼지고기를 살짝 볶아주세요.
3. 김치를 넣고 함께 볶아주세요.
4. 물을 붓고 끓이면서 간을 맞춰주세요.
5. 두부와 대파를 넣고 마무리합니다.

팁: 김치는 잘 익은 것을 사용하면 더욱 맛있습니다.

맛있게 드세요!
----------------------------------------

Role: user
-----------------
Text: 이 레시피에 들이가는 영양성분을 알려주세요
----------------------------------------

Role: assistant
-----------------
Text: 김치 찌개의 주요 영양성분을 상세히 알려드리겠습니다.

🥬 영양성분 (1인분 기준):
- 칼로리: 약 250-300kcal
- 단백질: 15-20g
- 지방: 15-18g
- 탄수화물: 10-15g

🌟 주요 영양소:
1. 단백질 
- 돼지고기: 근육 발달, 면역력 강화
- 두부: 식물성 단백질 공급

2. 비타민
- 비타민 A, C: 면역력 증진
- 비타민 B군: 대사 활성화

3. 무기질
- 칼슘: 뼈 건강
- 철분: 혈액 순환
- 칼륨: 혈압 조절

4. 김치의 특별한 영양
- 프로바이오틱스: 장 건강 개선
- 항산화 성분: 노화 방지

건강하고 맛있는 한 끼 식사가 되시길 바랍니다!
-----------------------------

### 3.4 Converse stream API 사용 
- Strem 형식의 출력을 사용

In [74]:
# 앞선 메세지에 3번째 질문을 추가한다. 
messages.append(  
   {
        "role": "user",
        "content": [{"text": "지금까지 내용을 정리해줘"}]
    }
)

response = bedrock.converse_stream(
        modelId=base_model,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

# Strem 형식의 출력 
stream = response.get('stream')
if stream:
    for event in stream:

        if 'messageStart' in event:
            print(f"\nRole: {event['messageStart']['role']}")

        if 'contentBlockDelta' in event:
            print(event['contentBlockDelta']['delta']['text'], end="")

        if 'messageStop' in event:
            print(f"\nStop reason: {event['messageStop']['stopReason']}")

        if 'metadata' in event:
            metadata = event['metadata']
            if 'usage' in metadata:
                print("\nToken usage")
                print(f"Input tokens: {metadata['usage']['inputTokens']}")
                print(
                    f":Output tokens: {metadata['usage']['outputTokens']}")
                print(f":Total tokens: {metadata['usage']['totalTokens']}")
            if 'metrics' in event['metadata']:
                print(
                    f"Latency: {metadata['metrics']['latencyMs']} milliseconds")



Role: assistant
김치 찌개 요리 가이드 요약

📋 레시피
재료:
- 김치 1컵
- 돼지고기 200g
- 두부 1모
- 대파 1대
- 고춧가루, 국간장, 다진 마늘

조리 순서:
1. 김치 자르기
2. 돼지고기 볶기
3. 김치와 함께 볶기
4. 물 붓고 끓이기
5. 두부, 대파 넣고 마무리

🥬 영양성분 (1인분 기준)
- 칼로리: 250-300kcal
- 단백질: 15-20g
- 지방: 15-18g
- 탄수화물: 10-15g

주요 영양소:
- 단백질: 근육 발달
- 비타민: 면역력 증진
- 무기질: 뼈 건강, 혈액 순환
- 김치: 장 건강, 항산화 효과

맛있고 건강한 한식 요리를 즐기세요!
Stop reason: end_turn

Token usage
Input tokens: 795
:Output tokens: 333
:Total tokens: 1128
Latency: 4901 milliseconds


## 4. 프롬프트 종류

### 4.1 프롬프트 실행을 위한 Model 함수 (Helper 함수) 생성

In [75]:
def ModelInvoke(user_messages, system_prompt="", inference_config={"temperature": 0.5}, additional_model_fields={"top_k": 200},modelId=base_model):
    # 메시지 구성
    messages = [{
        "role": "user", 
        "content": [{"text": user_messages}]
    }]

    # 시스템 프롬프트가 있는 경우에만 추가
    if system_prompt:
        system_prompts = [{"text": system_prompt}]
        response = bedrock.converse_stream(
            modelId=modelId,
            messages=messages,
            system=system_prompts,
            inferenceConfig=inference_config,
            additionalModelRequestFields=additional_model_fields
        )
    else:
        response = bedrock.converse_stream(
            modelId=modelId, 
            messages=messages,
            inferenceConfig=inference_config,
            additionalModelRequestFields=additional_model_fields
        )

    # Stream 형식의 출력
    stream = response.get('stream')
    if stream:
        for event in stream:

            if 'messageStart' in event:
                print(f"\nRole: {event['messageStart']['role']}")

            if 'contentBlockDelta' in event:
                print(event['contentBlockDelta']['delta']['text'], end="")

            if 'messageStop' in event:
                print(f"\nStop reason: {event['messageStop']['stopReason']}")

            if 'metadata' in event:
                print("\n---------- MetaData --------------")
                metadata = event['metadata']

                if 'usage' in metadata:
                    print(f"Input tokens: {metadata['usage']['inputTokens']}")
                    print(f"Output tokens: {metadata['usage']['outputTokens']}")
                    print(f"Total tokens: {metadata['usage']['totalTokens']}")
                if 'metrics' in event['metadata']:
                    print(f"Latency: {metadata['metrics']['latencyMs']} milliseconds")
                print("---------- MetaData --------------")



### 4.3 System prompt를 함께 사용

In [76]:
ModelInvoke(user_messages="짜장면 요리법 알려줘",
            system_prompt="중식당 사장님으로써 성실한 조언을 해야 합니다.")


Role: assistant
짜장면 만드는 방법을 알려드리겠습니다:

재료:
- 면
- 돼지고기
- 양파
- 대파
- 춘장
- 전분
- 설탕
- 간장

조리 순서:
1. 돼지고기를 깍둑썰기로 자르기
2. 양파도 같은 크기로 썰기
3. 팬에 기름 두르고 돼지고기 볶기
4. 양파 넣고 함께 볶기
5. 춘장 넣고 볶기
6. 물과 전분 섞어 소스 농도 맞추기
7. 삶은 면 위에 소스 얹어내기

팁: 
- 소스는 중간 불에서 천천히 볶기
- 면은 살짝 단단하게 삶기

맛있게 드세요!
Stop reason: end_turn

---------- MetaData --------------
Input tokens: 47
Output tokens: 278
Total tokens: 325
Latency: 5217 milliseconds
---------- MetaData --------------


### 4.4 Inference parameter를 함께 사용함

- temperature 값을 1 로 높일수록 다양한 아이디어를 생성하게 된다.

In [77]:
ModelInvoke(user_messages="토끼와 거북이가 경주하는 모습을 한문장에 6줄이내로 표현해줘!",
            system_prompt="유치원 선생님으로써 친절하게 이야기해줘!",
            inference_config={"temperature": 1.0},
            additional_model_fields={"top_k": 100})


Role: assistant
여기 선생님이 도와드릴게요! 

🐰🐢 토끼와 거북이 경주 이야기 🏃‍♂️

느림보 거북이와 자신만만한 토끼가 숲속 작은 길에서 승부를 겨루는데, 토끼는 앞서 달리다 자만심에 잠들고 성실한 거북이는 꾸준히 달려 결국 승리를 거머쥐었답니다.
Stop reason: end_turn

---------- MetaData --------------
Input tokens: 68
Output tokens: 154
Total tokens: 222
Latency: 3907 milliseconds
---------- MetaData --------------


In [78]:
ModelInvoke(user_messages="LLM에서 temperature를 활용한 예문을 알려줘!",
            inference_config={"temperature": 0.1},
            additional_model_fields={"top_k": 100})


Role: assistant
LLM의 temperature 파라미터는 출력의 무작위성과 창의성을 조절하는 중요한 설정입니다. 다음은 temperature 활용 예시입니다:

1. 낮은 Temperature (0.1-0.3)
- 보다 일관되고 결정론적인 응답
- 사실적이고 정확한 정보 생성
- 학술 논문, 기술 문서 작성에 적합

예시:
```python
response = model.generate(
    prompt="Python의 리스트 정렬 방법을 설명해줘",
    temperature=0.2  # 정확하고 표준적인 설명
)
```

2. 중간 Temperature (0.5-0.7)
- 균형 잡힌 창의성과 일관성
- 일반적인 대화나 콘텐츠 생성에 적합

예시:
```python
response = model.generate(
    prompt="여름 휴가로 추천하는 여행지",
    temperature=0.6  # 다양하고 흥미로운 제안
)
```

3. 높은 Temperature (0.8-1.0)
- 매우 창의적이고 다양한 응답
- 브레인스토밍, 창작 글쓰기에 적합

예시:
```python
response = model.generate(
    prompt="SF 단편소설의 독창적인 줄거리",
    temperature=0.9  # 매우 독특하고 창의적인 아이디어
)
```

이렇게 temperature를 조절하여 원하는 출력 스타일을 얻을 수 있습니다.
Stop reason: end_turn

---------- MetaData --------------
Input tokens: 29
Output tokens: 490
Total tokens: 519
Latency: 8618 milliseconds
---------- MetaData --------------


## 5. 도구 사용

### 5.1 Tool 선언 

In [79]:
from datetime import datetime

# 현재 시간을 반환하는 함수 정의 (Funtion Tool)
def get_current_time():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


### 5.2 Tool 과 Messagess를 Model에게 전달 

In [80]:
# Tool을 정의함.
tools = [
    {
        "toolSpec": {
            "name": "get_current_time",
            "description": "현재 시간을 반환합니다",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    }
]

# 메시지 정의
messages = [
    {
        "role": "user",
        "content": [{"text": "현재 시간이 몇 시인가요?"}]
    }
]

# Converse API 호출 하면서 Tool 요청 
response = bedrock.converse(
    modelId=base_model,
    messages=messages,
    toolConfig={"tools": tools}
)

# 응답 처리
output_message = response['output']['message']
print("------------------ Tool 요청이 있는 메세지, 'toolUse' 확인 ----------------------------------")
print(output_message)
print("---------------------------------------------------------------------------------------")


------------------ Tool 요청이 있는 메세지, 'toolUse' 확인 ----------------------------------
{'role': 'assistant', 'content': [{'text': '현재 시간을 확인하겠습니다.'}, {'toolUse': {'toolUseId': 'tooluse_Ff189nxSSW6_ESBKCiH_fA', 'name': 'get_current_time', 'input': {}}}]}
---------------------------------------------------------------------------------------


### 5.2 모델의 Tool사용 요청을 받아 Tool실행후 모델에게 전달 

In [81]:
# 모델의 응답으로 부터 Tool 사용('toolUse') 요청이 있는지 확인하고 tool요청이 있으면 funtion tool을 실행한다. 
if 'content' in output_message:
    for content in output_message['content']:
        if 'toolUse' in content:
            tool_use = content['toolUse']
            tool_name = tool_use['name']

            # 현재 시간을 확인하는 Funtion Tool 실행
            if tool_name == "get_current_time":
                current_time = get_current_time()

                # 이전 대화 목록에 Tool 실행(현재날짜와 시간정보)결과를 메시지에 추가
                messages.append(output_message)
                messages.append({
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use['toolUseId'],
                                "content": [{"text": current_time}]
                            }
                        }
                    ]
                })

                # 모델에게 Tool 실행 결과와 대화기록을 함께 전달한다. 
                final_response = bedrock.converse(
                    modelId=base_model,
                    messages=messages,
                    toolConfig={"tools": tools}
                )

                # 모델의 최종 응답 출력
                final_content = final_response['output']['message']['content'][0]['text']
                print(final_content)
        else:
            # Tool 사용 없이 직접 응답한 경우
            print(content['text'])

print("\n------------------ 전체 대화 내용 확인  ----------------------------------")
for item in messages:
    print(item)
print("\n")
print("---------------------------------------------------------------------------------------")


현재 시간을 확인하겠습니다.
현재 시간은 21시 05분 13초입니다.

------------------ 전체 대화 내용 확인  ----------------------------------
{'role': 'user', 'content': [{'text': '현재 시간이 몇 시인가요?'}]}
{'role': 'assistant', 'content': [{'text': '현재 시간을 확인하겠습니다.'}, {'toolUse': {'toolUseId': 'tooluse_Ff189nxSSW6_ESBKCiH_fA', 'name': 'get_current_time', 'input': {}}}]}
{'role': 'user', 'content': [{'toolResult': {'toolUseId': 'tooluse_Ff189nxSSW6_ESBKCiH_fA', 'content': [{'text': '2025-09-28 21:05:13'}]}}]}


---------------------------------------------------------------------------------------
